In [1]:
from ipywidgets import interact, Dropdown, Output, GridspecLayout, VBox, BoundedIntText, HBox, Button, TwoByTwoLayout, Layout
import calculations

### Define the Devourer Level widgets

In [2]:
# Define the options for dropdown menus
options = {
    'Devourer Lvl': [x for x in range(35, 101, 5)]
}

# Define the output textbox
output_level = Output()

# Define a function to update the level textbox
def update_output_level(dev_level_dropdown_val):
    output_level.clear_output()
    with output_level:
        print(f"{calculations.get_level_shards(dev_level_dropdown_val):,}")    # :, automatically adds the thousands separator

# Create dropdown widgets
dev_level_dropdown = Dropdown(options=options['Devourer Lvl'], description='Devourer Lvl:')

# Create an interaction between the dropdowns and the output textbox
interact(update_output_level,
         dev_level_dropdown_val=dev_level_dropdown);

interactive(children=(Dropdown(description='Devourer Lvl:', options=(35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 8…

### Define the Feats widgets

In [3]:
# Define the options for dropdown menus
options = {
    'Feats': [x for x in range(1, 26)],
}

# Define the output textbox
output_feats = Output()

# Define a function to update the feats textbox
def update_output_feats(feats_dropdown_val):
    output_feats.clear_output()
    with output_feats:
        print(f"{calculations.get_feat_shards(feats_dropdown_val)}%")

# Create dropdown widgets
feats_dropdown = Dropdown(options=options['Feats'], description='Feats:')

# Create an interaction between the dropdowns and the output textbox
interact(update_output_feats,
         feats_dropdown_val=feats_dropdown);

interactive(children=(Dropdown(description='Feats:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

### Define the Other widget

In [4]:
##### Define the output textbox
output_other = Output()

# Define int input box for Other category
other_shards = BoundedIntText(
    value=100,
    min=100,
    description='Other:',
    disabled=False
)

# Define a function to update the other textbox
def update_output_other(other_val):
    output_other.clear_output()
    with output_other:
        print(f"{other_val}%")

# Create an interaction between the dropdowns and the output textbox
interact(update_output_other,
         other_val=other_shards);

interactive(children=(BoundedIntText(value=100, description='Other:', min=100), Output()), _dom_classes=('widg…

### Define the legendaries widget

In [13]:
# Define the output textbox
output_legend = Output()

# Define the options for dropdown menus
options = calculations.list_legendary_limits()

# Define the descriptions for dropdown menus
descriptions = calculations.list_legendaries()

# Create a 4x4 grid to hold the dropdowns
grid = GridspecLayout(4, 4)

# Create callback for grid changes
def cell_eventhandler(change):
    output_legend.clear_output()
    with output_legend:
        calculations.update_legendary_count(change['new'], change['owner'].description)
        print(calculations.get_legendary_shards())

# Create callback for clear button
def on_clear_click(click):
    calculations.init_legendary_table()
    reset_grid()


# Reset legendary counts to 0
def reset_grid():
    for i in range(4):
        for j in range(3):
            grid[i, j].value = 0

# Loop through each cell in the grid
for i in range(4):
    for j in range(3):
        idx = (3 * i) + j
        desc = descriptions[idx]
        
        if options[idx] == -1:
            textbox = BoundedIntText(value=0, min=0, description=f'{desc}', disabled=False)
            textbox.observe(cell_eventhandler, names='value')
            grid[i, j] = textbox
        else:
            dropdown = Dropdown(options=[x for x in range(0, options[idx] + 1)], description=f'{desc}')
            dropdown.observe(cell_eventhandler, names='value')
            grid[i, j] = dropdown

# Add output box and clear button in last column
clear_button = Button(description='Clear', disabled=False, layout=Layout(height='auto', width='auto'))
grid[:4, 3] = clear_button

# Add callback to button
clear_button.on_click(on_clear_click)

### Put it all together

In [14]:
level_and_feat = TwoByTwoLayout(
    top_left=dev_level_dropdown, 
    top_right=output_level,
    bottom_left=feats_dropdown,
    bottom_right=output_feats)
other = HBox([other_shards, output_other])

widget = VBox([dev_level_dropdown, feats_dropdown, grid, other_shards])